<a href="https://colab.research.google.com/github/Pravin71/Gru-Network/blob/master/Chatter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project Plan

1. Clean the data:

 1.1 Topic labels - find major categories, remove invalid catogories (foreign language, non category...), one hot encoding
 
 1.2 Question, answer - combine both fields, word embedding
2. Model building

 2.1 Start with traditional algorithm
 
 2.2 Test Neural Networks
 
 2.3. RNN
 
3. Model tuning

In [0]:
from numpy.random import seed
seed(8)

from tensorflow import set_random_seed
set_random_seed(8)

import numpy as np
seed = np.random.RandomState(8)

import urllib.request
import tarfile
import os
import re
import string
from time import time

import numpy as np
import pandas as pd
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!python -m spacy download en_core_web_md

    100% |████████████████████████████████| 120.9MB 50.1MB/s 
  Running setup.py install for en-core-web-md ... - \ | / - \ | / - \ done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



In [0]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

## Exploring the Data

In [0]:
path = './gdrive/My Drive/WorkingDir/Chatter'
file_name = 'challenge.csv'

df = pd.read_csv(os.path.join(path, file_name))
df.head()

,company_id,survey_id,question_id,question_text,reply_id,reply_answer,topic_name,topic_sentiment,topic_question_dependent,question_asked_at
0,27,37,484,Almost done. Is there anything we could do to ...,3611,reduce wait times,Service:Wait Time,0.0,False,2017-05-17 19:37:54 UTC
1,27,37,473,Last question. Do you have any comments or fee...,3780,overall great customer service,Service:Customer Service,1.0,False,2017-05-18 14:09:47 UTC
2,27,37,476,We're sorry to hear that. Would you mind shari...,3595,i still need to help with my transponder,Service:Issue Being Resolved,-1.0,False,2017-05-17 19:34:52 UTC
3,27,37,476,We're sorry to hear that. Would you mind shari...,3751,i don't agree with the bill i got,Pricing,0.0,False,2017-05-18 12:24:03 UTC
4,27,37,476,We're sorry to hear that. Would you mind shari...,3776,faster shipping,Shipping:Speed,-0.5,False,2017-05-18 14:09:05 UTC


In [0]:
df.shape

(103047, 10)

In [0]:
df['topic_name'].str.contains(':')

0          True
1          True
2          True
3         False
4          True
5          True
6         False
7          True
8          True
9          True
10         True
11         True
12         True
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
103017     True
103018     True
103019    False
103020    False
103021    False
103022     True
103023    False
103024     True
103025     True
103026     True
103027     True
103028    False
103029     True
103030    False
103031     True
103032     True
103033     True
103034     True
103035    False
103036     True
103037     True
103038     True
103039     True
103040     True
103041     True
103042     True
103043     True
103044     True
103045     True
103046     True
Name: topic_name, Length

In [0]:
df.dtypes

company_id                    int64
survey_id                     int64
question_id                   int64
question_text                object
reply_id                      int64
reply_answer                 object
topic_name                   object
topic_sentiment             float64
topic_question_dependent       bool
question_asked_at            object
dtype: object

In [0]:
df.topic_name.value_counts

<bound method IndexOpsMixin.value_counts of 0                           Service:Wait Time
1                    Service:Customer Service
2                Service:Issue Being Resolved
3                                     Pricing
4                              Shipping:Speed
5                              Staff:Rudeness
6                          Company Perception
7                             Brand:Awareness
8                             Brand:Awareness
9                             Brand:Awareness
10                            Brand:Awareness
11                            Brand:Awareness
12                            Brand:Awareness
13                          Previous Purchase
14                          Previous Purchase
15                          Previous Purchase
16                          Previous Purchase
17                          Previous Purchase
18                          Previous Purchase
19                          Previous Purchase
20                                  

In [0]:
df.topic_name.unique()

array(['Service:Wait Time', 'Service:Customer Service',
       'Service:Issue Being Resolved', 'Pricing', 'Shipping:Speed',
       'Staff:Rudeness', 'Company Perception', 'Brand:Awareness',
       'Previous Purchase', 'Product', 'Company:Advertising',
       'Store Experience:Layout', 'Product:Variety', 'Product:Sample',
       'Staff', 'Store Experience:Customer Suggestions',
       'Product:Flavour', 'Staff:Persistence', 'Product:Smell',
       'Store Experience:Music', 'Store Experience:Purchase After Demo',
       'Product:Size', 'Staff:Helpful', 'Service:General',
       'Company:Promotions & Sales', 'Staff:Friendly',
       'Service:Product Specific Knowledge', 'Store Experience:Signage',
       'Product:Labelling', 'Service:Speed of Service',
       'Product:Availability', 'Store Experience:Speed of Checkout',
       'Store Experience', 'Store Experience:Size',
       'Store Experience:Proximity', 'Store Experience:Lack of Time',
       'Service:Online Store', 'Product:Uniquenes

In [0]:
len(df.topic_name.unique())

545

In [0]:
df.topic_name.unique().tolist()

['Service:Wait Time',
 'Service:Customer Service',
 'Service:Issue Being Resolved',
 'Pricing',
 'Shipping:Speed',
 'Staff:Rudeness',
 'Company Perception',
 'Brand:Awareness',
 'Previous Purchase',
 'Product',
 'Company:Advertising',
 'Store Experience:Layout',
 'Product:Variety',
 'Product:Sample',
 'Staff',
 'Store Experience:Customer Suggestions',
 'Product:Flavour',
 'Staff:Persistence',
 'Product:Smell',
 'Store Experience:Music',
 'Store Experience:Purchase After Demo',
 'Product:Size',
 'Staff:Helpful',
 'Service:General',
 'Company:Promotions & Sales',
 'Staff:Friendly',
 'Service:Product Specific Knowledge',
 'Store Experience:Signage',
 'Product:Labelling',
 'Service:Speed of Service',
 'Product:Availability',
 'Store Experience:Speed of Checkout',
 'Store Experience',
 'Store Experience:Size',
 'Store Experience:Proximity',
 'Store Experience:Lack of Time',
 'Service:Online Store',
 'Product:Uniqueness',
 'Product Demo',
 'Service:Specific Person Stood Out',
 'Store Experie

## Preprocessing the Data

In [0]:
df_sample = df.sample(frac=0.01)
df_sample.shape

(1030, 10)

In [0]:
df_sample.head()

,company_id,survey_id,question_id,question_text,reply_id,reply_answer,topic_name,topic_sentiment,topic_question_dependent,question_asked_at
57437,33,316,5655,"And this one too :\r\n\r\n""The best part about...",1050406,that i know they're safe products,Product,1.0,True,2018-09-06 05:45:07 UTC
95384,6,302,5328,Please complete the following sentence:\r\n\r\...,1418055,Lots of toys to choose from - and Julie,Product:Variety,1.0,False,2018-10-07 23:07:51 UTC
61593,33,316,5655,"And this one too :\r\n\r\n""The best part about...",1257704,All the samples,Product:Sample,1.0,False,2018-09-23 20:54:36 UTC
41629,33,316,5655,"And this one too :\r\n\r\n""The best part about...",534813,that everything is all natural and vegan and m...,Product:Organic,1.0,False,2018-08-03 06:02:59 UTC
48448,33,316,5655,"And this one too :\r\n\r\n""The best part about...",743717,the all natural products and the customer serv...,Product:Natural,1.0,True,2018-08-15 18:11:21 UTC


In [0]:
df_sample['category'] = df_sample['topic_name'].apply(lambda x: str(x).split(':')[0] if ':' in str(x) else x)
df_sample.head(10)

,company_id,survey_id,question_id,question_text,reply_id,reply_answer,topic_name,topic_sentiment,topic_question_dependent,question_asked_at,category
57437,33,316,5655,"And this one too :\r\n\r\n""The best part about...",1050406,that i know they're safe products,Product,1.0,True,2018-09-06 05:45:07 UTC,Product
95384,6,302,5328,Please complete the following sentence:\r\n\r\...,1418055,Lots of toys to choose from - and Julie,Product:Variety,1.0,False,2018-10-07 23:07:51 UTC,Product
61593,33,316,5655,"And this one too :\r\n\r\n""The best part about...",1257704,All the samples,Product:Sample,1.0,False,2018-09-23 20:54:36 UTC,Product
41629,33,316,5655,"And this one too :\r\n\r\n""The best part about...",534813,that everything is all natural and vegan and m...,Product:Organic,1.0,False,2018-08-03 06:02:59 UTC,Product
48448,33,316,5655,"And this one too :\r\n\r\n""The best part about...",743717,the all natural products and the customer serv...,Product:Natural,1.0,True,2018-08-15 18:11:21 UTC,Product
75874,33,91,1400,What did you think of the demonstration?,107583,"I really liked it, the bathbomb was so pretty ...",Product:Smell,1.0,False,2017-12-17 18:10:44 UTC,Product
37005,33,316,5654,Please complete the following sentence :\r\n\r...,1129436,add more lavender products,Product:Variety,0.0,False,2018-09-12 22:16:46 UTC,Product
6472,33,313,5584,Almost done! Please complete the following sen...,490146,carry older products again,Online Experience:Product Availability,0.0,False,2018-07-31 13:08:05 UTC,Online Experience
87350,6,302,5328,Please complete the following sentence:\r\n\r\...,467954,the great staff,Staff,1.0,True,2018-07-28 01:23:15 UTC,Staff
34318,33,316,5654,Please complete the following sentence :\r\n\r...,996051,the pushiness of sales reps,Staff:Persistence,-0.5,False,2018-09-01 22:02:59 UTC,Staff


In [0]:
df_sample.category.unique()

array(['Product', 'Online Experience', 'Staff', 'Game Experience',
       'Shipping', 'Store Experience', 'Company', 'Product Demo',
       'Service', 'Store', 'Pricing', 'Previous Purchase', 'Charity', nan,
       'Campaign Idea', 'Price of Shipping', 'Company Brand',
       'Banking Experience', 'Phone Agent', 'Previous Visit', 'Hardware',
       'otherlanguage', 'Workshop Experience', 'Fan Experience'],
      dtype=object)

In [0]:
df_sample.query('category!="otherlanguage"', inplace=True)
df_sample.shape

(1029, 11)

In [0]:
df_sample.category.isnull().sum()

7

In [0]:
df_sample[df_sample.category.isnull()]

,company_id,survey_id,question_id,question_text,reply_id,reply_answer,topic_name,topic_sentiment,topic_question_dependent,question_asked_at,category
14593,33,314,5618,Almost done. Please complete the following sen...,602736,nothing it’s a nice site and easy to use.,NaN,0.0,False,2018-08-06 18:35:25 UTC,NaN
13199,33,314,5618,Almost done. Please complete the following sen...,457687,nothing so far!! great experience for the firs...,NaN,0.0,False,2018-07-26 19:53:47 UTC,NaN
2196,66,401,6680,"And this one too:\r\n\r\n""The one thing I wish...",1404076,Birthday 㳟舀,NaN,0.0,False,2018-10-07 00:58:33 UTC,NaN
5427,46,268,4746,Please complete the following sentence:\r\n\r\...,779498,i don't know,NaN,0.0,False,2018-08-17 22:10:10 UTC,NaN
27544,33,316,5654,Please complete the following sentence :\r\n\r...,674859,"nothing, we love lush",NaN,0.0,False,2018-08-11 02:41:54 UTC,NaN
36774,33,316,5654,Please complete the following sentence :\r\n\r...,1115928,the ladies in the crossiron mills mall calgary...,NaN,0.0,False,2018-09-11 14:14:34 UTC,NaN
13748,33,314,5618,Almost done. Please complete the following sen...,488699,nothing :),NaN,0.0,False,2018-07-31 02:53:30 UTC,NaN


In [0]:
df_sample.dropna(subset=['category'], inplace=True)
df_sample.shape

(1022, 11)

In [0]:
df_sample.head()

,company_id,survey_id,question_id,question_text,reply_id,reply_answer,topic_name,topic_sentiment,topic_question_dependent,question_asked_at,category
57437,33,316,5655,"And this one too :\r\n\r\n""The best part about...",1050406,that i know they're safe products,Product,1.0,True,2018-09-06 05:45:07 UTC,Product
95384,6,302,5328,Please complete the following sentence:\r\n\r\...,1418055,Lots of toys to choose from - and Julie,Product:Variety,1.0,False,2018-10-07 23:07:51 UTC,Product
61593,33,316,5655,"And this one too :\r\n\r\n""The best part about...",1257704,All the samples,Product:Sample,1.0,False,2018-09-23 20:54:36 UTC,Product
41629,33,316,5655,"And this one too :\r\n\r\n""The best part about...",534813,that everything is all natural and vegan and m...,Product:Organic,1.0,False,2018-08-03 06:02:59 UTC,Product
48448,33,316,5655,"And this one too :\r\n\r\n""The best part about...",743717,the all natural products and the customer serv...,Product:Natural,1.0,True,2018-08-15 18:11:21 UTC,Product


In [0]:
# Combine question and answer columns
df_sample['question_answer'] = df_sample['question_text'] + ' ' + df_sample['reply_answer']
df_sample.head()

,company_id,survey_id,question_id,question_text,reply_id,reply_answer,topic_name,topic_sentiment,topic_question_dependent,question_asked_at,category,question_answer
57437,33,316,5655,"And this one too :\r\n\r\n""The best part about...",1050406,that i know they're safe products,Product,1.0,True,2018-09-06 05:45:07 UTC,Product,"And this one too :\r\n\r\n""The best part about..."
95384,6,302,5328,Please complete the following sentence:\r\n\r\...,1418055,Lots of toys to choose from - and Julie,Product:Variety,1.0,False,2018-10-07 23:07:51 UTC,Product,Please complete the following sentence:\r\n\r\...
61593,33,316,5655,"And this one too :\r\n\r\n""The best part about...",1257704,All the samples,Product:Sample,1.0,False,2018-09-23 20:54:36 UTC,Product,"And this one too :\r\n\r\n""The best part about..."
41629,33,316,5655,"And this one too :\r\n\r\n""The best part about...",534813,that everything is all natural and vegan and m...,Product:Organic,1.0,False,2018-08-03 06:02:59 UTC,Product,"And this one too :\r\n\r\n""The best part about..."
48448,33,316,5655,"And this one too :\r\n\r\n""The best part about...",743717,the all natural products and the customer serv...,Product:Natural,1.0,True,2018-08-15 18:11:21 UTC,Product,"And this one too :\r\n\r\n""The best part about..."


In [0]:
def preprocess_text(text, lemmatizing=False):
    """
    Preprocesses text by removing all stop words and lemmatizing.
    
    Params
    --------------
    text: str
      the string to be cleaned
    
    Returns
    --------------
    text: str
      the cleaned string

    """
    text = text.lower()
    
    if lemmatizing:
      lemmatizer = WordNetLemmatizer()
      text = ' '.join([lemmatizer.lemmatize(word, pos='v') for word in text.split()])
    
    # Replace punctuation with spaces
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    text = text.translate(translator)

    # Remove stop words
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    
    # Remove additional white spaces
    text = ' '.join(text.split())
    
    return text

In [0]:
cols_to_drop = [col for col in df_sample.columns.tolist() if col not in ['question_answer', 'category']]
df_sample.drop(columns=cols_to_drop, inplace=True)
df_sample.head()

,category,question_answer
57437,Product,"And this one too :\r\n\r\n""The best part about..."
95384,Product,Please complete the following sentence:\r\n\r\...
61593,Product,"And this one too :\r\n\r\n""The best part about..."
41629,Product,"And this one too :\r\n\r\n""The best part about..."
48448,Product,"And this one too :\r\n\r\n""The best part about..."


In [0]:
df_sample['question_answer_clean'] = df_sample['question_answer'].apply(preprocess_text)

In [0]:
df_sample['question_answer'].iloc[10]

'Please complete the following sentence:\r\n\r\n"The best part about Purdys is..." dipped ice cream bars'

In [0]:
df_sample['question_answer_clean'].iloc[10]

'please complete following sentence best part purdys dipped ice cream bars'

## Traing the Model

In [0]:
nlp = spacy.load('en_core_web_md')
X_train = np.array(list(df_sample['question_answer'].apply(lambda x: nlp(x).vector.tolist())))

In [0]:
X_train.shape

(1022, 300)

In [0]:
df_sample.head()

,category,question_answer,question_answer_clean
57437,Product,"And this one too :\r\n\r\n""The best part about...",one best part shopping lush know safe products
95384,Product,Please complete the following sentence:\r\n\r\...,please complete following sentence best thing ...
61593,Product,"And this one too :\r\n\r\n""The best part about...",one best part shopping lush samples
41629,Product,"And this one too :\r\n\r\n""The best part about...",one best part shopping lush everything natural...
48448,Product,"And this one too :\r\n\r\n""The best part about...",one best part shopping lush natural products c...


In [0]:
y_train = pd.get_dummies(df_sample.category)
y_train.shape

(1022, 22)

In [0]:
from keras.optimizers import RMSprop

model = Sequential()
model.add(Dense(64, input_dim=300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(22, activation='softmax'))

#adam = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=100,
          batch_size=128)

Epoch 1/100
1022/1022 [==============================] - 1s 952us/step - loss: 2.9191 - acc: 0.1693
Epoch 2/100
1022/1022 [==============================] - 0s 33us/step - loss: 2.6167 - acc: 0.3180
Epoch 3/100
1022/1022 [==============================] - 0s 33us/step - loss: 2.4307 - acc: 0.3376
Epoch 4/100
1022/1022 [==============================] - 0s 37us/step - loss: 2.2745 - acc: 0.3425
Epoch 5/100
1022/1022 [==============================] - 0s 45us/step - loss: 2.2098 - acc: 0.3317
Epoch 6/100
1022/1022 [==============================] - 0s 39us/step - loss: 2.1600 - acc: 0.3239
Epoch 7/100
1022/1022 [==============================] - 0s 45us/step - loss: 2.1470 - acc: 0.3356
Epoch 8/100
1022/1022 [==============================] - 0s 42us/step - loss: 2.0707 - acc: 0.3337
Epoch 9/100
1022/1022 [==============================] - 0s 40us/step - loss: 2.0278 - acc: 0.3708
Epoch 10/100
1022/1022 [==============================] - 0s 51us/step - loss: 2.0060 - acc: 0.3591
Epoch 11

## Working on the Full Dataset

In [0]:
def preprocess_data(df):
  df['category'] = df['topic_name'].apply(lambda x: str(x).split(':')[0] if ':' in str(x) else x)
  df.query('category!="otherlanguage"', inplace=True)
  df.dropna(subset=['category'], inplace=True)
  df['question_answer'] = df['question_text'] + ' ' + df['reply_answer']
  df['question_answer_clean'] = df['question_answer'].apply(preprocess_text)
  
  cols_to_drop = [col for col in df_sample.columns.tolist() if col not in ['question_answer', 'category']]
  df.drop(columns=cols_to_drop, inplace=True)
  
  X = np.array(list(df['question_answer'].apply(lambda x: nlp(x).vector.tolist())))
  y = pd.get_dummies(df.category)
  
  return X, y 

In [0]:
X, y = preprocess_data(df)

In [0]:
model = Sequential()
model.add(Dense(64, input_dim=300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(61, activation='softmax'))

#adam = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y,
          epochs=100,
          batch_size=128)

Epoch 1/100
101914/101914 [==============================] - 4s 37us/step - loss: 1.6203 - acc: 0.5225
Epoch 2/100
101914/101914 [==============================] - 4s 36us/step - loss: 1.0392 - acc: 0.6859
Epoch 3/100
101914/101914 [==============================] - 4s 36us/step - loss: 0.9128 - acc: 0.7273
Epoch 4/100
101914/101914 [==============================] - 4s 36us/step - loss: 0.8403 - acc: 0.7474
Epoch 5/100
101914/101914 [==============================] - 4s 36us/step - loss: 0.7955 - acc: 0.7581
Epoch 6/100
101914/101914 [==============================] - 4s 36us/step - loss: 0.7628 - acc: 0.7666
Epoch 7/100
101914/101914 [==============================] - 4s 37us/step - loss: 0.7398 - acc: 0.7702
Epoch 8/100
101914/101914 [==============================] - 4s 37us/step - loss: 0.7200 - acc: 0.7751
Epoch 9/100
101914/101914 [==============================] - 4s 36us/step - loss: 0.7076 - acc: 0.7786
Epoch 10/100
101914/101914 [==============================] - 4s 37us/ste